### Download dataset and annotations
1) annotations: http://images.cocodataset.org/annotations/annotations_trainval2017.zip
2) sample dataset (240mb) https://datasetninja.com/coco-2017#download


In [2]:
from pycocotools.coco import COCO
import os

In [4]:
coco = COCO('annotations/instances_train2017.json')

loading annotations into memory...
Done (t=19.69s)
creating index...
index created!


#### Electrodomésticos

* Tv, 4768
* Laptop, 3707
* Mouse, 1964
* Keyboard, 2221
* Remote, 3221
* Cellphone, 5017
* Microwave, 1601
* Toaster, 225
* Refrigerator, 2461
* Oven, 2992

### Filter images for the selected categories

In [92]:
categories = ["laptop", "keyboard", "mouse", "keyboard", "cell phone", "tv"]

# this is necessary because the method getImgIds performs AND instead of OR
img_ids = []
for cat in categories:
    img_ids += coco.getImgIds(catIds=coco.getCatIds(catNms=[cat]))

# remove duplicates
img_ids = list(set(img_ids))

print(f"Found {len(img_ids)} images that contain at least one of the selected categories.")

Found 11354 images that contain at least one of the selected categories.


In [108]:
import os
import shutil

if not os.path.exists('filtered_images'):
    os.makedirs('filtered_images/images')

In [109]:
annotations_dir = 'filtered_images/annotations'
os.makedirs(annotations_dir, exist_ok=True)

In [110]:
# create new dict to hold the filtered annotations

filtered_annotations = {
    'info': coco.dataset['info'],
    'licenses': coco.dataset['licenses'],
    'images': [],
    'annotations': [],
    'categories': [cat for cat in coco.dataset['categories'] if cat['id'] in cat_ids]
}

In [ ]:
# filter images and their corresponding annotations
for img_id in img_ids:
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join('train2017/img', img_info['file_name'])

    if os.path.exists(img_path):
        shutil.copy(img_path, 'filtered_images/images')
        filtered_annotations['images'].append(img_info)
        
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_ids)
        anns = coco.loadAnns(ann_ids)
        for ann in anns:
            filtered_annotations['annotations'].append(ann)
        
        filtered_count += 1
    else:
        print(f"Image file {img_path} not found.")

annotation_file_path = os.path.join('filtered_images/annotations', 'instances_filtered.json')
with open(annotation_file_path, 'w') as f:
    json.dump(filtered_annotations, f)